In [ ]:
import pandas as pd
import numpy as np
import re
from itertools import product

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.metrics import mean_squared_error, mean_squared_log_error

from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold, cross_val_score, RandomizedSearchCV, TimeSeriesSplit
from sklearn.feature_selection import mutual_info_classif

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

from sklearn.cluster import KMeans

from scipy.stats import loguniform, uniform
from random import randrange
from statsmodels.tsa.seasonal import seasonal_decompose

import torch
from torch.utils.data import TensorDataset, DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
data_dir = '/kaggle/input/competitive-data-science-predict-future-sales/'
raw_df = pd.read_csv(data_dir + 'sales_train.csv')
raw_df['date'] = pd.to_datetime(raw_df['date'], format="%d.%m.%Y")
test_df = pd.read_csv(data_dir + 'test.csv')
sub_df = pd.read_csv(data_dir + 'sample_submission.csv')
items_df = pd.read_csv(data_dir + 'items.csv')
item_cat_df = pd.read_csv(data_dir + 'item_categories.csv')
shops_df = pd.read_csv(data_dir + 'shops.csv')

In [ ]:
import plotly.express as px

df_plot = raw_df.copy()
df_plot = df_plot[['date','item_cnt_day']].groupby(['date']).sum().reset_index()
df_plot['month'] = df_plot['date'].dt.to_period('M').apply(lambda r: r.start_time)
df_plot['month_num'] = df_plot['month'].rank(method="dense").astype(int)
df_plot


In [ ]:
raw_df

In [ ]:
plt.plot(df_plot['date'], df_plot['item_cnt_day'], label='Total Sales')
plt.xticks(rotation=45)  # Xoay nhãn trục x 45 độ

In [ ]:
item_cat_df

In [ ]:
items_df

In [ ]:
item_cat_df

In [ ]:
shops_df

In [ ]:
test_df

In [ ]:
plt.figure(figsize=(12,8))
plt.xlim(-100,3000) ## tạo 1 đồ thì có 1 trục x
flierprops = dict(marker='o', markerfacecolor='purple', markersize=6,
                  linestyle='none', markeredgecolor='black') # các điểm trên đồ thị biểu diễn bởi 'o' màu tím
sns.boxplot(x=raw_df['item_cnt_day'], flierprops=flierprops)
# => đồ thị biểu diễn phân bố sản phầm bán đc trong 1 ngày 

plt.figure(figsize=(12,8))
plt.xlim(-50,4e5)
sns.boxplot(x=raw_df['item_price'], flierprops=flierprops)
# => đồ thị biểu diễn phân bố giá

In [ ]:
# raw_df = raw_df[(raw_df['item_price']<300000)&(raw_df['item_cnt_day']<1000)]
# lọc bỏ các giá trị bất thường giá trị quá cao hoặc quá thấp
raw_df[(raw_df['item_price']<300000)&(raw_df['item_cnt_day']<1000)].count()
raw_df[(raw_df['item_price']>300000)|(raw_df['item_cnt_day']>1000)].count()

In [ ]:
# raw_df = raw_df[raw_df['item_price']>0].reset_index(drop=True) # loại bỉ các sản phẩm có giá âm
# raw_df['item_cnt_day'] = raw_df['item_cnt_day'].apply(lambda x: 0 if x<1 else x) # đưa các ngày có giá trị âm thành 0
a= raw_df[raw_df['item_price']<0]['date'].reset_index(drop=True).count()
b = raw_df[raw_df['item_cnt_day']<0]['date'].count()
print(a, b)

### Cleaning Shop Data

In [ ]:
shops_df

In [ ]:
shops_df['shop_city'] = shops_df['shop_name'].apply(lambda x: str(x).split(' ')[0]) # cắt thành phố và địa tên ra riêng
shops_df['shop_category'] = shops_df['shop_name'].apply(lambda x: str(x).split(' ')[1]) 
# And simple a city
shops_df['shop_city'] = shops_df['shop_city'].apply(lambda x: 'Якутск' if x=='!Якутск' else x)
shops_df

### Cleaning Item Data

In [ ]:
items_df

In [ ]:
def correct_item(x): ## làm sạch và xử lí đồng bộ các loại mặt hàng
    x = x.lower() # LowerCase
    x = x.partition('[')[0]
    x = x.partition('(')[0]
    x = re.sub(r'[^A-Za-z0-9А-Яа-я]+',' ',x) # remove special char
    x = x.replace('  ', ' ')
    x = x.strip()
    return x

In [ ]:
# chia dữ tên thành 3 thành phần
items_df['item_name1'] = items_df['item_name'].apply(lambda x: re.split(r'[\[\(]',x,1)[0])
items_df['item_name2'] = items_df['item_name'].apply(lambda x: str(x).split('[',1)[1] if str(x).find('[')>=0 else '0')
items_df['item_name3'] = items_df['item_name'].apply(lambda x: str(x).split('(',1)[1] if str(x).find('(')>=0 else '0')

In [ ]:
items_df['item_name2'] = items_df['item_name2'].apply(lambda x: '0' if x=='' else x) # nếu itemname2 là rỗng thì chuyển thành 0

In [ ]:
items_df['item_type'] = items_df['item_name2'].apply(lambda x: x[0:8]
                                                     if x.split(' ')[0]=='xbox'
                                                     else x.split(' ')[0]) # nếu tên là xbox thì chỉ lấy 8 kí tự thôi chắc dài quá
items_df['item_type'].unique()

In [ ]:
## 1 số loại mặt hàng gần giống nhau cho vào 1 loại
def clean_item_type(x):
    if x=='xbox 360' or x=='x360' or x=='xbox360':
        return 'x360'
    if x=='pc' or x=='pс' or x=='рс':
        return 'pc'
    if x=='ps3' or x=='рs3':
        return 'ps3'
    return x
items_df['item_type'] = items_df['item_type'].apply(lambda x: clean_item_type(x))

In [ ]:
## lại làm giống trên lọc lấy các mặt hàng xuất hiện >20
item_types = items_df['item_type'].value_counts()
item_types = item_types[item_types>=20]
item_types_idx = item_types.index.tolist()
items_df['item_type'] = items_df['item_type'].apply(lambda x: x if x in item_types_idx else 'rem')

In [ ]:
## lại làm giống trên lọc lấy các mặt hàng xuất hiện >20
item_types = items_df['item_type'].value_counts()
item_types = item_types[item_types>=20]
item_types_idx = item_types.index.tolist()
items_df['item_type'] = items_df['item_type'].apply(lambda x: x if x in item_types_idx else 'rem')

### Cleaning Shop Data

In [ ]:
shops_df

In [ ]:
shops_df['shop_city'] = shops_df['shop_name'].apply(lambda x: str(x).split(' ')[0]) # cắt thành phố và địa tên ra riêng
shops_df['shop_category'] = shops_df['shop_name'].apply(lambda x: str(x).split(' ')[1]) 
# And simple a city
shops_df['shop_city'] = shops_df['shop_city'].apply(lambda x: 'Якутск' if x=='!Якутск' else x)

In [ ]:
shop_cats = shops_df['shop_category'].value_counts() # tạo 1 bảng riêng đếm số lần xuất hiện của các cửa hàng
shop_cats = shop_cats[shop_cats>=5] # trích lấy những cửa hàng có số lần xuất hiện lớn hơn 5
shop_cats_idx = shop_cats.index.tolist() # tạo 1 danh sách các cửa hàng có số lần xuất hiện lớn hơn 5
shops_df['shop_category'] = shops_df['shop_category'].apply(lambda x: x if x in shop_cats_idx else 'rem')  # giữ lại các cử hàng xuất hiện >5 lần còn lại chuyển thành 'rem'

In [ ]:
encoder = OrdinalEncoder() # chọn encoder là ordinalEncoder (chuyển từ dạng số thành dạng categorical)
shops_df[['shop_city','shop_category']] = encoder.fit_transform(shops_df[['shop_city','shop_category']]) # chuyển shop_city và shop_category thành categorical

In [ ]:
shops_df = shops_df[['shop_id','shop_city','shop_category']]

### Cleaning Item Category Data

In [ ]:
item_cat_df

In [ ]:
item_cat_df['item_category_type'] = item_cat_df['item_category_name'].apply(lambda x: str(x).split(' ')[0])  # lấy loại sản phẩm 
item_cat_df['item_category_type'] = item_cat_df['item_category_type'].apply(lambda x: 'Игры' 
                                                                            if (x=='Игровые') or (x=='Аксессуары') 
                                                                            else x)

In [ ]:
# làm tương tự như bên cửa hàng
item_cat_types = item_cat_df['item_category_type'].value_counts()
item_cat_types = item_cat_types[item_cat_types>=5]
item_cat_types_idx = item_cat_types.index.tolist()
item_cat_df['item_category_type'] = item_cat_df['item_category_type'].apply(lambda x: x if x in item_cat_types_idx else 'rem')

In [ ]:
item_cat_df['split_sub'] = item_cat_df['item_category_name'].apply(lambda x: str(x).split('-'))
item_cat_df['item_category_sub_type'] = item_cat_df['split_sub'].apply(lambda x: x[1].strip() if len(x)>1 else x[0].strip())

In [ ]:
encoder = OrdinalEncoder()
item_cat_df[['item_category_type','item_category_sub_type']] = encoder.fit_transform(item_cat_df[['item_category_type','item_category_sub_type']])

In [ ]:
item_cat_df = item_cat_df[['item_category_id','item_category_type','item_category_sub_type']]

## Feature Engineering and Preprocessing

In [ ]:
raw_df['revenue'] = raw_df['item_cnt_day'] * raw_df['item_price'] ## tạo thêm feature doanh thu

In [ ]:
cols = ['date_block_num','shop_id','item_id']

In [ ]:
matrix = []
cols = ['date_block_num','shop_id','item_id']
for i in range(34):
    sales = raw_df[raw_df['date_block_num']==i]
    matrix.append(np.array(list(product([i],sales['shop_id'].unique(),sales['item_id'].unique())), dtype=np.int16))

matrix_df = pd.DataFrame(data=np.vstack(matrix), columns=cols)
matrix_df["date_block_num"] = matrix_df["date_block_num"].astype('int8')
matrix_df["shop_id"] = matrix_df["shop_id"].astype('int8')
matrix_df["item_id"] = matrix_df["item_id"].astype('int16')
matrix_df.sort_values(cols, inplace=True)
### không cần qua tâm code chỉ hiểu là tạo 1 chứa tất cả các cặp shop-item từ trước đến nay

In [ ]:
group = raw_df.groupby(cols)['item_cnt_day'].aggregate(['sum'])
group.columns = ['item_cnt_month']
group.reset_index(inplace=True)
group
## tạo cái bảng dưới @@
matrix_df = matrix_df.reset_index(drop=True)
matrix_df = pd.merge(matrix_df, group, on=cols, how='left')
matrix_df.fillna({'item_cnt_month':0},inplace=True)
matrix_df['item_cnt_month'] = matrix_df['item_cnt_month'].astype('float16')

In [ ]:
test_df['date_block_num'] = 34 ## chỉnh tháng của các sản phẩm cần dự đoán là 34
test_df['date_block_num'] = test_df['date_block_num'].astype('int8')
test_df['shop_id'] = test_df['shop_id'].astype('int8')
test_df['item_id'] = test_df['item_id'].astype('int16')
## chỉnh kiểu dữ liệu 
# test_df.drop(columns='ID',inplace=True) đã drop rồi nên không có nữa
test_df

In [ ]:
concat_df = pd.concat([matrix_df,test_df]).copy()
concat_df = concat_df.reset_index(drop=True)
concat_df.fillna({'item_cnt_month':0},inplace=True)
concat_df
## hiểu là ghép test_df và nối tiếp matrix

In [ ]:
concat_df = pd.merge(concat_df, items_df, on='item_id', how='left')
concat_df = pd.merge(concat_df, item_cat_df, on='item_category_id', how='left')
concat_df = pd.merge(concat_df, shops_df, on='shop_id', how='left')
concat_df

In [ ]:
items_df['item_name2'] = items_df['item_name2'].apply(lambda x: '0' if x=='' else x) # nếu itemname2 là rỗng thì chuyển thành 0

In [ ]:
concat_df.columns

In [ ]:
concat_df = concat_df.drop(columns=['item_name1','item_name'])

In [ ]:
for col in ['item_category_id','item_type','item_category_type',
            'item_category_sub_type','shop_city','shop_category']:
    concat_df[col] = concat_df[col].astype('int8')
concat_df['item_name2'] = concat_df['item_name2'].astype('int16')
concat_df['item_name3'] = concat_df['item_name3'].astype('int16')
## chỉnh hết các cột dữ liệu float thành int16, int8

## Training model

In [ ]:
concat_df

In [ ]:
concat_df.drop(columns=['ID'], inplace=True)

In [ ]:
def create_lag_feature(df, lags, features):
    for feature in features:
        print(feature)
        for lag in lags:
            shifted = df[['date_block_num','shop_id','item_id',feature]].copy()
            shifted.columns = ['date_block_num','shop_id','item_id',f'{feature}_lag_{lag}']
            shifted['date_block_num'] = shifted['date_block_num'] + lag
            df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df
    
lag_for_ml = [i+1 for i in range(3)]
concat_df = create_lag_feature(concat_df, lags=lag_for_ml, features=['item_cnt_month'])

In [ ]:
# concat_df = create_sequences(df = concat_df,features= ['item_cnt_month'] ,sequence_length = 5)

In [ ]:
train_inputs = concat_df[(concat_df['date_block_num']<33)&(concat_df['date_block_num']>3)]
train_targets = concat_df[(concat_df['date_block_num']<33)&(concat_df['date_block_num']>3)]['item_cnt_month']

val_inputs = concat_df[concat_df['date_block_num']==33]
val_targets = concat_df[concat_df['date_block_num']==33]['item_cnt_month']

test_inputs = concat_df[concat_df['date_block_num']==34]

In [ ]:
train_targets = train_targets.clip(0,20)
val_targets = val_targets.clip(0,20)

In [ ]:
xgb_param =  {'colsample_bytree': 0.8406368905921987, 'device': 'cuda', 'eval_metric': 'rmse', 'learning_rate': 0.023622088178310074, 'max_bin': 256, 'max_depth': 12, 'min_child_weight': 1.9311905509233425, 'n_estimators': 700, 'nthread': 2, 'reg_lambda': 0.004641439538226732, 'subsample': 0.8055867639368266, 'tree_method': 'hist'}

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor  # Đảm bảo import đúng

In [ ]:
xgb_param['objective'] = 'reg:squarederror'  # Hoặc 'reg:absoluteerror'
model = xgb.XGBRegressor(**xgb_param)

In [ ]:
eval_set = (val_inputs, val_targets)
model.fit(train_inputs, train_targets, eval_set=[eval_set], verbose=False)

In [ ]:
train_preds = model.predict(train_inputs)
val_preds = model.predict(val_inputs) 
train_score = np.sqrt(mean_squared_error(train_targets, train_preds))
val_score = np.sqrt(mean_squared_error(val_targets, val_preds))

In [ ]:
train_score, val_score

In [ ]:
test_preds = model.predict(test_inputs) 

In [ ]:
sub_df['item_cnt_month'] = test_preds.clip(0,20)
sub_df.to_csv('/kaggle/working/submission.csv', index=None)
sub_df